In [ ]:

import os
import subprocess
from pathlib import Path
from tqdm.auto import tqdm

def check_ffmpeg():
    try:
        subprocess.run(["ffmpeg", "-version"], stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
    except Exception as e:
        raise RuntimeError("FFmpeg not found. Please install FFmpeg and add to PATH.") from e

check_ffmpeg()
print("FFmpeg is available.")

def compress_video(
    input_path: Path,
    output_path: Path,
    *,
    codec: str = "libx265",
    crf: int = 28,
    preset: str = "medium",
    audio_codec: str = "aac",
    audio_bitrate: str = "128k"
) -> None:
    """
    Compress a video file using FFmpeg.

    Parameters:
      input_path    Path to source video
      output_path   Path to write compressed video
      codec         Video codec: libx265, libvpx-vp9, libaom-av1
      crf           Constant Rate Factor (lower is higher quality)
      preset        Encoding preset: ultrafast, fast, medium, slow, etc.
      audio_codec   Audio codec: aac, libopus, etc.
      audio_bitrate Audio bitrate, e.g. '128k'
    """
    cmd = [
        "ffmpeg",
        "-y",  # overwrite
        "-i", str(input_path),
        "-c:v", codec,
        "-crf", str(crf),
        "-preset", preset,
        "-c:a", audio_codec,
        "-b:a", audio_bitrate,
        str(output_path)
    ]
    # Run with progress
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, bufsize=1)
    for line in process.stdout:
        if "frame=" in line or "size=" in line:
            tqdm.write(line.strip())
    process.wait()
    if process.returncode != 0:
        raise RuntimeError(f"FFmpeg failed (code {process.returncode})")

def batch_compress(
    src_dir: Path,
    dst_dir: Path,
    compress_kwargs
):
    dst_dir.mkdir(parents=True, exist_ok=True)
    videos = list(src_dir.glob("*.mp4"))  # adjust pattern if needed
    for vid in tqdm(videos, desc="Compressing"):
        out_file = dst_dir / f"{vid.stem}_compressed{vid.suffix}"
        compress_video(vid, out_file, compress_kwargs)
        orig_sz = vid.stat().st_size / (10243)
        new_sz  = out_file.stat().st_size / (10243)
        print(f"{vid.name}: {orig_sz:.2f} GB → {new_sz:.2f} GB")